In [1]:
import os
import sys
sys.path.append("/home/pervinco/Upstage_Ai_Lab/Final/IR/src")

import faiss
import huggingface_hub
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from dotenv import load_dotenv
load_dotenv("../keys.env")

upstage_api_key = os.getenv("UPSTAGE_API_KEY")
os.environ['UPSTAGE_API_KEY'] = upstage_api_key

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

voyage_api_key = os.getenv('VOYAGE_API_KEY')
os.environ['VOYAGE_API_KEY'] = voyage_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

from langchain_chroma import Chroma
from langchain.storage import InMemoryByteStore
from langchain.retrievers.multi_vector import SearchType
from langchain.retrievers.multi_vector import MultiVectorRetriever

from config import Args
from dense_retriever.model import load_upstage_encoder, load_hf_encoder
from dense_retriever.doc_processor import score_normalizer
from data.data import load_document, chunking, load_hypothetical_quries, load_document_summary

/home/pervinco/miniconda3/envs/ir-project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/pervinco/.cache/huggingface/token
Login successful
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/pervinco/.config/sagemaker/config.yaml


In [2]:
args = Args()

chunks = load_document(args.doc_file_path)
summaries = load_document_summary("../dataset/summarized_documents.jsonl")
hy_quries = load_hypothetical_quries("../dataset/processed_documents_queries.jsonl")

print(len(chunks))
print(len(summaries))
print(len(hy_quries))

24799
4272
12816


In [3]:
persist_directory = "./index_files/chroma/upstage/chunk"

embedder = load_hf_encoder(args.hf_model_name, args.model_kwargs, args.encode_kwargs) ## load_upstage_encoder(args.upstage_model_name)
store = InMemoryByteStore()

if not os.path.exists(persist_directory):
    vectorstore = Chroma(collection_name="cr_chunks", embedding_function=embedder, persist_directory=persist_directory)
    vectorstore.add_documents(chunks, persist_directory=persist_directory)

else:
    vectorstore = Chroma(persist_directory=persist_directory)

In [4]:
id_key = "docid"
retriever = MultiVectorRetriever(vectorstore=vectorstore,
                                 byte_store=store,
                                 id_key=id_key)

retriever.search_type = SearchType.mmr ## mmr, similarity, similarity_score_threshold

In [5]:
print(len(vectorstore.get()['documents']))

24799


In [6]:
retriever.vectorstore.similarity_search("복숭아 키우는 노하우좀?")

[Document(metadata={'docid': 'bf023a9d-47f3-4d77-8bf1-5a854b5403a7'}, page_content='통해 복숭아나무는 더욱 강건한 복숭아를 생산할 수 있습니다. 이러한 복제 방법은 복숭아 생산자들에게 매우 중요한 기술이며, 복숭아의 품질과 수량을 향상시키는 데 큰 도움이\n\n이 청크는 복숭아나무의 병 저항성 복숭아 생산과 관련된 농업 기술 문서에서 발췌되었습니다. 이 문서에서는 복숭아나무가 무성 생식을 통해 병에 강한 복숭아를 생산하는 방법을 설명하고 있으며, 이러한 복제 기술이 복숭아 생산자들에게 품질과 수량을 향상시키는 데 중요한 역할을 한다고 강조하고 있습니다. 청크는 특히 복숭아나무의 강건한 복숭아 생산과 복제 방법의 중요성에 대한 부분에서 발췌되었습니다.'),
 Document(metadata={'docid': 'bf023a9d-47f3-4d77-8bf1-5a854b5403a7'}, page_content='특정 복숭아를 정확하게 복제하는 방법은 복숭아나무가 무성 생식을 하도록 하는 것입니다. 복숭아나무는 꽃이 핀 후에 수술을 통해 수술한 꽃을 다른 복숭아나무에 이식하여 교배를\n\n이 청크는 복숭아나무의 병 저항성 향상과 관련된 농업 기술 문서에서 발췌되었습니다. 이 문서에서는 특정 병에 강한 복숭아를 복제하는 방법으로 무성 생식을 통한 교배 기술을 설명하고 있습니다. 특히, 복숭아나무의 꽃을 수술하여 다른 나무에 이식하는 과정을 통해 병 저항성을 가진 복숭아를 생산하는 방법에 초점을 맞추고 있습니다.'),
 Document(metadata={'docid': 'bf023a9d-47f3-4d77-8bf1-5a854b5403a7'}, page_content='복숭아나무는 다른 복숭아보다 병에 대한 저항력이 더 큰 복숭아를 생산합니다. 이러한 특정 복숭아를 정확하게 복제하는 방법은 복숭아나무가 무성 생식을 하도록 하는 것입니다.\n\n이 청크는 복숭아 재배와 관련된 농업 기술 문서에서 발췌되었습니다. 이

In [7]:
persist_directory = "./index_files/chroma/upstage/chunk_summary"
retriever.vectorstore.add_documents(summaries, persist_directory=persist_directory)

['739a6c38-5ae9-467e-a414-0f4c7272ce5c',
 '013266ac-52e2-4193-84c1-bae83ccf2446',
 '87480458-80f2-42c5-95f6-5a52c49b8213',
 'c5b46217-d090-421e-b041-a0738220ce5b',
 'ec72aaf1-0459-4d8f-966f-76e00950fc05',
 '4120808b-0046-45b9-b29b-0809dbf0ee14',
 '5f1f623c-6d46-40df-a372-03c6788bb225',
 'e1e7e52c-e860-4d57-9a41-35b0fab5adbd',
 'f9993cb9-06d5-4b2a-ae1d-0192cf6d7c2d',
 '2a14a1c3-564e-4cab-8b57-83badb968f44',
 'bab507e6-f7a7-4566-821d-ad50e8a2c932',
 '29a45230-1d6c-4a4d-b2d7-9ec384772165',
 'd02cee35-1d9d-4c36-a790-6a8caf829923',
 '943309a7-3cea-4d7d-81ac-e3cf0bda0941',
 '8ff29540-069e-4c31-ab6f-7d9559d23980',
 'f6aabdfb-0efc-4044-8a07-c8d6b4aab6c5',
 '729228a4-ff8f-445e-a8a8-180146086daa',
 '254468e6-b054-44c8-b048-e3803e1dc861',
 '7466ed41-7f9c-4477-9307-9728aa7e7e46',
 '1d8597fe-2e4b-47c2-aa15-d952a16d5b4a',
 'e7fdaa16-3f67-455b-92ff-06db35adf7fd',
 'd2631e90-8abb-4b58-9089-18e9c4d10001',
 'd7ebe97b-563c-46a0-ae17-047d4b75c2f3',
 '29eb7a3a-78be-403f-af72-60246d95aed6',
 'fef44e24-02fc-

In [8]:
print(len(vectorstore.get()['documents']))

29071


In [9]:
retriever.vectorstore.similarity_search("복숭아 키우는 노하우좀?")

[Document(metadata={'docid': 'bf023a9d-47f3-4d77-8bf1-5a854b5403a7'}, page_content='통해 복숭아나무는 더욱 강건한 복숭아를 생산할 수 있습니다. 이러한 복제 방법은 복숭아 생산자들에게 매우 중요한 기술이며, 복숭아의 품질과 수량을 향상시키는 데 큰 도움이\n\n이 청크는 복숭아나무의 병 저항성 복숭아 생산과 관련된 농업 기술 문서에서 발췌되었습니다. 이 문서에서는 복숭아나무가 무성 생식을 통해 병에 강한 복숭아를 생산하는 방법을 설명하고 있으며, 이러한 복제 기술이 복숭아 생산자들에게 품질과 수량을 향상시키는 데 중요한 역할을 한다고 강조하고 있습니다. 청크는 특히 복숭아나무의 강건한 복숭아 생산과 복제 방법의 중요성에 대한 부분에서 발췌되었습니다.'),
 Document(metadata={'docid': 'bf023a9d-47f3-4d77-8bf1-5a854b5403a7'}, page_content='특정 복숭아를 정확하게 복제하는 방법은 복숭아나무가 무성 생식을 하도록 하는 것입니다. 복숭아나무는 꽃이 핀 후에 수술을 통해 수술한 꽃을 다른 복숭아나무에 이식하여 교배를\n\n이 청크는 복숭아나무의 병 저항성 향상과 관련된 농업 기술 문서에서 발췌되었습니다. 이 문서에서는 특정 병에 강한 복숭아를 복제하는 방법으로 무성 생식을 통한 교배 기술을 설명하고 있습니다. 특히, 복숭아나무의 꽃을 수술하여 다른 나무에 이식하는 과정을 통해 병 저항성을 가진 복숭아를 생산하는 방법에 초점을 맞추고 있습니다.'),
 Document(metadata={'docid': 'bf023a9d-47f3-4d77-8bf1-5a854b5403a7'}, page_content='복숭아나무는 다른 복숭아보다 병에 대한 저항력이 더 큰 복숭아를 생산합니다. 이러한 특정 복숭아를 정확하게 복제하는 방법은 복숭아나무가 무성 생식을 하도록 하는 것입니다.\n\n이 청크는 복숭아 재배와 관련된 농업 기술 문서에서 발췌되었습니다. 이

In [10]:
persist_directory = "./index_files/chroma/upstage/chunk_summary_hypqury"
retriever.vectorstore.add_documents(hy_quries, persist_directory=persist_directory)

['900a2565-68df-4950-91cc-95f3a9a35a4d',
 '8db2fcec-9c7f-4a19-a453-74f4e6e7bc0e',
 '7485546a-446d-40a9-8685-8288bfb6b397',
 '3f508de1-9eb8-47de-9528-13fb88017d6d',
 'c72c5f0b-e5b6-45d2-83a0-89cfc2d8ba24',
 'bfe57554-0156-4860-90ff-1c867f1f937e',
 'ce111435-ce2c-4d0f-bbec-6b91c9367376',
 '6c5d6947-cb00-4d99-93d1-e51710432afb',
 '111df929-fdaa-4107-8be2-8ef9d6b33d72',
 '08c62c53-332a-434c-98a4-40cac8c9b475',
 '0d5641cb-d46c-4c52-ad50-6020821d2e33',
 'd8d37c1a-f336-4fb9-aacb-39191b8dbeca',
 '87fe58e7-7285-48be-8ab3-2db2e8c3f4de',
 '9ea2fd8b-37db-46b3-a2ac-f9299f16ee92',
 'bdf96ecf-449a-4660-9f2c-8b62aa912b88',
 '46fa4bc6-555b-49a3-8baf-7508f233144c',
 '35b526d0-2423-4aa2-b539-e61f6c916e16',
 'a975fee5-2c8b-48b7-ad19-8d90de0dd1e0',
 'e4b909b3-129f-4e0f-bf03-dea138e9044e',
 'f71c6982-ee46-498b-a464-ac653319b38e',
 '1f4bfca7-2317-488b-aa6f-312eea64fb7d',
 '86e24388-4a9d-481e-ad3b-b12c0ee3e8f5',
 '1ae48e77-2c05-4b33-b0a0-680c2fbe3d13',
 '64548808-470c-41bf-8177-610dab98a8eb',
 '65a00f85-3cfe-

In [17]:
print(len(vectorstore.get()['documents']))

41887


In [15]:
test = Chroma(persist_directory="./index_files/chroma/upstage/chunk", collection_name="cr_chunks")

In [16]:
print(len(test.get()['documents']))

41887
